In [1]:

import csv
import random
from datetime import datetime, timedelta
import time
import pytz
from pytz import timezone
from pydantic import BaseModel
import pymongo
from pymongo import MongoClient
import requests
from pushbullet import Pushbullet
import pandas as pd
from csv import writer
import stockFormula
import talib
import pandas_ta as ta
import os.path

In [2]:

def RSI(df):
    close = df['close']
    rsi = talib.RSI(close, timeperiod=14)
    rsi2 = talib.RSI(close, timeperiod=14)
    df['rsi'] = rsi
    df['rsi2'] = rsi2
    return df

     
def RSIMovingAverage(list,index,divident):
    data = list[:index+1]
    data = data[-divident:]
    sum=0.00
    for i in data:
        sum=sum+ RSI(list,list.index(i),divident)
    return(sum/divident)

def MovingAverage(df):
    df['MA200'] = df['close'].rolling(200).mean()
    df['MA100'] = df['close'].rolling(100).mean()
    df['MA50'] = df['close'].rolling(50).mean()
    df['MA20'] = df['close'].rolling(20).mean()
    df['MA14'] = df['close'].rolling(14).mean()
    df['MA5'] = df['close'].rolling(5).mean()
    return df

def AverageVolume(df) :
    df['av']  = df['volume'].rolling(30).mean()
    return df
       
def hikendataconvertion(data):
    hikendata =  data.copy()
    for i in data[1:] :
         prevdata=hikendata[data.index(i)-1].split(",")
         arrdata=data[data.index(i)].split(",")
         HAopen=round(((float(prevdata[1]) + float(prevdata[4]))/2),2)
         HAclose=round(((float(arrdata[1])+float(arrdata[4])+float(arrdata[2])+float(arrdata[3]))/4),2)
         testdata=arrdata[0]+","+str(round(((float(prevdata[1]) + float(prevdata[4]))/2),2))+","+str(max(HAopen,HAclose,float(arrdata[2])))+","+str(min(HAopen,HAclose,float(arrdata[3])))+","+str(round(((float(arrdata[1])+float(arrdata[4])+float(arrdata[2])+float(arrdata[3]))/4),2))+","+arrdata[5]
         hikendata[data.index(i)]=testdata
    return hikendata

def convertToDF(data):
    return  pd.DataFrame({
                    'date':data["t"],
                    'open': data["o"],
                    'high': data["h"],
                    'low':data["l"],
                    'close':data["c"],
                    'volume':data["v"]})
                    
def VWMA(df):
    df['vwma'] = df.ta.vwma(length = 20,fillna=True)
    return df

def VA(df,type):
    if type=="1WVA":
         df['1WVA'] = df['volume'].rolling(5).mean()
    return df
        

    
def MACD(df):
    # Get the 26-day EMA of the closing price
    k = df['close'].ewm(span=12, adjust=False, min_periods=12).mean()
    # Get the 12-day EMA of the closing price
    d = df['close'].ewm(span=26, adjust=False, min_periods=26).mean()
    # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
    macd = k - d
    # Get the 9-Day EMA of the MACD for the Trigger line
    macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
    # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
    macd_h = macd - macd_s
    # Add all of our new values for the MACD to the dataframe
    df['macd'] = df.index.map(macd)
    df['macd_h'] = df.index.map(macd_h)
    df['macd_s'] = df.index.map(macd_s)

    return df

# responseday = requests.get("https://api.upstox.com/historical/NSE_EQ/1660/day?timestamp=")
# data=responseday.json()['data']
# print(data.index(data[-1]))
# index=data.index(data[-4])
# print (data[index])
# hikendata=hikendataconvertion(data)
# MovingAverage(hikendata,index,50)todate = datetime.today()
def heikin_ashi(df):
    heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['open', 'high', 'low', 'close'])
    
    heikin_ashi_df['close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    
    for i in range(len(df)):
        if i == 0:
            heikin_ashi_df.iat[0, 0] = df['open'].iloc[0]
        else:
            heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i-1, 0] + heikin_ashi_df.iat[i-1, 3]) / 2
        
    heikin_ashi_df['high'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['high']).max(axis=1)
    
    heikin_ashi_df['low'] = heikin_ashi_df.loc[:, ['open', 'close']].join(df['low']).min(axis=1)
    heikin_ashi_df['volume']=df['volume']
    heikin_ashi_df['date']=df['date']

    return heikin_ashi_df


def bollinger_bands(df):
    currunt_upper_bollinger_band = ta.bbands(df["close"], length=20, std=2)
    df['up']=currunt_upper_bollinger_band["BBU_20_2.0"]
    df['down']=currunt_upper_bollinger_band["BBL_20_2.0"]
    df['middle']=currunt_upper_bollinger_band["BBM_20_2.0"]
    return df
   
def ADX(df):
    df['adx']  = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    return df
    
def percentageCalc(num1,num2):
    x=abs(num1-num2)
    y=num1+num2
    z=x/y
    return z*100

def findPercentage(num1,num2):
    return int(num1)/int(num2)*100

In [3]:

def getStockData(stockCode,timeList):
    data=[]
    for timeLine in timeList:
        #todate = datetime.today()-timedelta(days=365)
        todate = datetime.today()
        fromdate = todate - timedelta(days=timeLine[1])
        response = requests.get("https://priceapi.moneycontrol.com/techCharts/indianMarket/stock/history?symbol="+stockCode+"&resolution="+timeLine[0]+"&from="+str(int (time.mktime(fromdate.timetuple())))+"&to="+str(int (time.mktime(todate.timetuple()))))
        data.append([timeLine[0],response.json()])
    return data
    
def getPreLoadedData(timeLine,stockCode):
    print(" processing from state")
    dfList=[]
    ha_dfList=[]
    for t in timeLine:
        df = pd.read_pickle("./DFState/data"+stockCode+t[0]+".pkl")
        ha_df=pd.read_pickle("./DFState/HAdata"+stockCode+t[0]+".pkl")
        dfList.append(df.copy())
        ha_dfList.append(ha_df.copy())
    return [dfList,ha_dfList]


def preProcess(dataList,stockCode):
    dfList=[]
    ha_dfList=[]
    for data in dataList:
        df=convertToDF(data[1])
        df=MACD(df)
        df=VWMA(df)
        df=RSI(df)
        df=VA(df,"1WVA")
        df=bollinger_bands(df)
        df=ADX(df)
        df=MovingAverage(df)
        df=AverageVolume(df)

        #df.to_pickle("./DFState/data"+stockCode+data[0]+".pkl")

        #hiken
        ha_df=heikin_ashi(df.copy())
        ha_df=MACD(ha_df)
        ha_df=VWMA(ha_df)
        ha_df=RSI(ha_df)
        ha_df=VA(ha_df,"1WVA")
        ha_df=bollinger_bands(ha_df)
        ha_df=ADX(ha_df)
        ha_df=MovingAverage(ha_df)
        ha_df=AverageVolume(ha_df)

        #df.to_pickle("./DFState/HAdata"+stockCode+data[0]+".pkl")
        dfList.append(df.copy())
        ha_dfList.append(ha_df.copy())
        
    return [dfList,ha_dfList]

def storeInFile(fileName,data):
    with open( fileName, 'a',newline='', encoding='utf-8') as object:
                                writer_object = writer(object)
                                writer_object.writerow(data)
                                object.close()

def calcWinWithPercentage(stratagy,overall,percentage):
    total=0
    win=0
    totalAmount=0
    winAmount=0
    for result in overall[stratagy]:
        total=total+1
        previous =0
        proceed=True
        totalAmount=totalAmount+10000
        #print("######################################")
        for i in range(6,15):
            currentAmount=10000+((10000/result[5])*result[i])
            #print("initial"+str(result[i]))
            if result[i]>0 and proceed:           
                if i==14:
                    #print(result[0]+"finalLoop"+str(result[i]))
                    winAmount=winAmount+currentAmount
                    #print(result[0]+"finalLoop@#################################"+str(winAmount))

                else:
                    current=percentageCalc(result[5],result[i]+result[5])             
                    if currentAmount-10000>100:
                        win=win+1
                        #print(result[0]+"possitive_Win"+str(result[i]))
                        winAmount=winAmount+currentAmount
                        #print(result[0]+"win_current_End@#################################"+str(winAmount))

                        proceed=False
                    elif previous>current:
                        #print(result[0]+"negative_current"+str(result[i]))
                        winAmount=winAmount+currentAmount
                        #print(result[0]+"negative_current_End@#################################"+str(winAmount))
                        proceed=False
                    previous=current
            else:
                if i==14 and  proceed:
                    #print(result[0]+"finalLoop_"+str(result[i]))
                    winAmount=winAmount+currentAmount
                    #print(result[0]+"finalLoop_@#################################"+str(winAmount))

                elif  proceed:
                    #print(result[0]+"negative_val"+str(result[i]))
                    winAmount=winAmount+currentAmount
                    #print(result[0]+"negative_val@#################################"+str(winAmount))

                break
    return [total,win,totalAmount,winAmount]

def findCandleType(df,index):
    if ((df.loc[:, 'open'][index]<df.loc[:, 'close'][index])and(df.loc[:, 'open'][index]==df.loc[:, 'low'][index]) ):
        return "G"
    elif ((df.loc[:, 'open'][index]>df.loc[:, 'close'][index])and(df.loc[:, 'open'][index]==df.loc[:, 'high'][index]) ):
        return "R"
    elif ((df.loc[:, 'open'][index]<df.loc[:, 'close'][index]) ):
        return "G1"
    else:
        return "R1"  

def isTouchLow(df,index):
    if((df.loc[:, 'close'][index]<df.loc[:, 'down'][index])):
        return True
    else:
        return False

def calcRealProfit(result,visited):
     count =(10000/float(result[6]))
     for i in range(7,16):
        if(float(result[i])*count>100):
            return True
        else:
            date=datetime.strptime(result[0], '%b %d %Y %I:%M%p')
            date=date+timedelta(days=i-5)
            visited.append(date.strftime("%b %d %Y %I:%M%p"))
     return False

def calcRealProfitInIsolation(result,stockVisited):
     count =(10000/float(result[6]))
     for i in range(7,16):
        if(float(result[i])*count>100):
            return True
        else:
            date=datetime.strptime(result[0], '%b %d %Y %I:%M%p')
            date=date+timedelta(days=i-5)
            stockVisited[result[2]].append(date.strftime("%b %d %Y %I:%M%p"))
     return False

def findRange(rangeList,val):
    for i in rangeList:
        if float(val)>float(i[0]) and float(val)<float(i[1]):
            return str(i[0])+"-"+str(i[1])
    return "null"    

def FCTP(df,index):
    if float(df.at[index, 'close']) > float(df.at[index, 'open']):
        return "G"
    else:
        return "R"  

In [4]:

with open('./Stocks/NSE_Test_Script_code.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0   
    timeLine=[["1D",665]]
    rootdf = pd.DataFrame()
    rootdfv = pd.DataFrame()
    wvadf = pd.DataFrame()
    adxdf = pd.DataFrame()
    rsidf = pd.DataFrame()

    stockRank=[]
    
    for row in csv_reader:
        try:
            print(row[1])
            dataList=getStockData( row[0],timeLine)
            #[dfList,ha_dfList]=getPreLoadedData(timeLine,row[0])
            [dfList,ha_dfList]=preProcess(dataList, row[0])
            dfList[0]['date'] = pd.to_datetime(dfList[0]['date'],unit='s')
            df=dfList[0][['date','close']]
            df=df.set_index('date')
            df.rename(columns = {'close':row[0]}, inplace = True)
            rootdf=pd.concat([rootdf, df], axis=1)

            # response = requests.get("https://api.tickertape.in/search?text="+row[0]+"&types=stock")
            # sid=response.json()["data"]["stocks"][0]["sid"]
            # #print("thirdtapi")
            # response = requests.get("https://api.tickertape.in/stocks/info/"+sid)
            # response=response.json()
            # if(int(response["data"]["ratios"]["mrktCapRank"])<100):
            #     stockRank.append(row[0])



            dfv=dfList[0][['date','volume']]
            dfv=dfv.set_index('date')
            dfv.rename(columns = {'volume':row[0]}, inplace = True)
            rootdfv=pd.concat([rootdfv, dfv], axis=1)

            wdf=dfList[0][['date','1WVA']]
            wdf=wdf.set_index('date')
            wdf.rename(columns = {'1WVA':row[0]}, inplace = True)
            wvadf=pd.concat([wvadf, wdf], axis=1)

            adf=dfList[0][['date','adx']]
            adf=adf.set_index('date')
            adf.rename(columns = {'adx':row[0]}, inplace = True)
            adxdf=pd.concat([adxdf, adf], axis=1)

            rdf=dfList[0][['date','rsi']]
            rdf=rdf.set_index('date')
            rdf.rename(columns = {'rsi':row[0]}, inplace = True)
            rsidf=pd.concat([rsidf, rdf], axis=1)

            line_count+= 1


            
        except Exception as e:
             print("Oops!", e, "occurred.")
    line_count += 1
    #rootdf['date'] = pd.to_datetime(rootdf['date'],unit='s')
    #rootdf =rootdf.dropna(axis=1)
    #mtl=(rootdf.pct_change()+1)[1:].resample('M').prod()
    #ret12,ret6,ret3=getRolling(mtl,12),getRolling(mtl,6),getRolling(mtl,3)

    print(f'Processed {line_count} lines.')



ACC Ltd
Aegis Logistics Ltd
Amrutanjan Health Care Ltd
Andhra Paper Ltd
Apollo Tyres Ltd
Arvind Ltd
Processed 7 lines.


In [8]:
rootdf =rootdf.dropna(axis=1)
#rootdf['date'] = pd.to_datetime(rootdf['date'],unit='s')
rootdf
mtl=(rootdf.pct_change()+1)[1:].resample('M').prod()
mtlv=(rootdfv)[1:].resample('M').prod()




,ACC,AEGISCHEM,AMRUTANJAN,ANDHRAPAP,APOLLOTYRE,ARVIND
date,,,,,,
2021-03-22,1.078205,0.937042,1.059111,1.001734,0.985118,0.963388
2021-03-25,0.983987,1.065055,1.058447,0.989829,1.013980,0.985842
2021-03-28,1.006499,0.965454,0.984083,0.973983,0.991550,1.006047
2021-03-31,1.020971,1.041487,1.040390,1.007407,1.058758,1.038317
2021-04-03,0.990096,0.956349,0.968039,0.980392,0.939420,0.959479
...,...,...,...,...,...,...
2022-12-30,0.997447,0.987923,1.012900,1.003823,0.993262,1.019329
2023-01-02,0.986771,0.986481,1.014143,0.996191,0.986895,0.970441
2023-01-05,1.002760,0.996793,0.983140,0.994863,1.024059,0.993103


In [6]:
import numpy as np

def get_rolling_ret(df,n):
    return df.rolling(n).apply(np.prod)

In [7]:
ret_12,ret_6,ret_3 = get_rolling_ret(mtl,12),get_rolling_ret(mtl,6),get_rolling_ret(mtl,3)
resv_12=get_rolling_ret(mtlv,12)

In [8]:
def get_top(date):    
    top_50=ret_12.loc[date].nlargest(50).index
    print(top_50)
    top_30=ret_6.loc[date,top_50].nlargest(30).index
    print(top_30)
    top_10=ret_3.loc[date,top_30].nlargest(10).index
    return top_10


In [24]:
rootdf =rootdf.dropna(axis=1)
mtl=(rootdf.pct_change()+1)[1:].resample('M').prod()

ret_12,ret_6,ret_3 = get_rolling_ret(mtl,12),get_rolling_ret(mtl,6),get_rolling_ret(mtl,3)


In [26]:
adx=adxdf.loc["2021-03-31"]


In [20]:
for  date in mtl.index[:-1]:
        print(date)
        stock_list=get_top(date)
        adx=adxdf.loc[date,stock_list]
        rsi=rsidf.loc[date,stock_list]
        wva=wvadf.loc[date,stock_list]

2021-03-31 00:00:00
Index([], dtype='object')
Index([], dtype='object')
2021-04-30 00:00:00
Index([], dtype='object')
Index([], dtype='object')
2021-05-31 00:00:00
Index([], dtype='object')
Index([], dtype='object')
2021-06-30 00:00:00
Index([], dtype='object')
Index([], dtype='object')
2021-07-31 00:00:00
Index([], dtype='object')
Index([], dtype='object')


KeyError: Timestamp('2021-07-31 00:00:00', freq='M')

In [49]:
top=resv_12.loc['2022-12-31'].nlargest(50).index
tips_filtered = ret_12.reindex(columns = stockRank)
tips_filtered
ret_12
tips_filtered = ret_12.reindex(columns = stockRank)
tips_filtered.loc['2022-12-31'].nlargest(100).index




Index(['ADANIPOWER', 'BANKBARODA', 'ADANIENT', 'VBL', 'HAL', 'UNIONBANK',
       'CANBK', 'TIINDIA', 'IOB', 'COALINDIA', 'JINDALSTEL', 'ITC', 'YESBANK',
       'ADANITRANS', 'ADANIGREEN', 'AMBUJACEM', 'AXISBANK', 'INDUSINDBK',
       'NTPC', 'EICHERMOT', 'ABB', 'SIEMENS', 'BRITANNIA', 'SUNPHARMA',
       'BHARTIARTL', 'IDBI', 'JSWSTEEL', 'CIPLA', 'ADANIPORTS', 'GAIL',
       'HEROMOTOCO', 'HDFCBANK', 'HINDUNILVR', 'RAILTEL', 'RELIANCE', 'GRASIM',
       'BAJAJHLDNG', 'POWERGRID', 'TATACONSUM', 'ONGC', 'SBILIFE', 'IOC',
       'TATASTEEL', 'HINDALCO', 'INDIGO', 'NESTLEIND', 'MARICO', 'DABUR',
       'DLF', 'SRF', 'TORNTPHARM', 'BAJAJFINSV', 'BAJFINANCE', 'TATAPOWER',
       'ULTRACEMCO', 'ASIANPAINT', 'VEDL', 'GODREJCP', 'APOLLOHOSP', 'ICICIGI',
       'HDFCLIFE', 'TCS', 'DMART', 'DRREDDY', 'SHREECEM', 'TATAMTRDVR',
       'SBICARD', 'ICICIPRULI', 'TATAMOTORS', 'HCLTECH', 'IRCTC', 'BERGEPAINT',
       'DIVISLAB', 'LTI', 'TECHM', 'WIPRO', 'WINPRO'],
      dtype='object')

In [27]:
portfolio=mtl.loc['2022-12-31':,'ANDHRAPAP'][1:2]
portfolio.values[0]

0.99428707450607

In [52]:
def pf_perfomance(date):
    portfolio = mtl.loc[date:,get_top(date)][1:2]
    return portfolio.mean(axis=1).values[0]

In [53]:
import matplotlib
returns=[]
for date in mtl.index[:-1]:
    returns.append(pf_perfomance(date))
    
pd.Series(returns,index=mtl.index[1:])

date
2021-04-30         NaN
2021-05-31         NaN
2021-06-30         NaN
2021-07-31         NaN
2021-08-31         NaN
2021-09-30         NaN
2021-10-31         NaN
2021-11-30         NaN
2021-12-31         NaN
2022-01-31         NaN
2022-02-28         NaN
2022-03-31    1.122700
2022-04-30    1.129381
2022-05-31    0.878518
2022-06-30    0.971378
2022-07-31    1.122468
2022-08-31    1.081877
2022-09-30    0.989608
2022-10-31    1.010482
2022-11-30    1.090905
2022-12-31    1.054834
2023-01-31    1.010997
Freq: M, dtype: float64

In [ ]:
for date in mtl.index[:-1]:
    stock_list=get_top(date)
    for stock in stock_list:
        #fromdate = datetime.strptime(date, '%Y-%m-%d')
        todate = datetime.today()
        diff=todate-date
        timeLine=[["1D",diff.days+1]]#becareful
        dataList=getStockData(stock,timeLine)
        df=stockFormula.convertToDF(dataList[0][1])

        open=df.at[1, 'open']
        flag="PROGRESS"
        for index, val in df.iterrows():
            if index>1 :
                if (10000/open)*(val['close']-open)>100: 
                    flag="PASS"
                    print(date,stock,"pass")
                    break
            if index==11:
                    flag="FAIL"
                    print(date,stock,"fail")
                    break
